In [1]:
pip install --upgrade geopandas fiona

Note: you may need to restart the kernel to use updated packages.


In [7]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import pandas as pd

# List of clusters with coordinates added
clusters = [
    {"name": "Lulea", "nodes": 6, "gpus_per_node": 2, "carbon-intensity-trace": "./carbon/SE-SE4.csv", "lat": 65.5848, "lon": 22.1547},
    {"name": "Stockholm", "nodes": 2, "gpus_per_node": 4, "carbon-intensity-trace": "./carbon/SE-SE3.csv", "lat": 59.3293, "lon": 18.0686},
    {"name": "Lund", "nodes": 3, "gpus_per_node": 4, "carbon-intensity-trace": "./carbon/SE-SE4.csv", "lat": 55.7047, "lon": 13.1910},
    {"name": "Amsterdam", "nodes": 3, "gpus_per_node": 6, "carbon-intensity-trace": "./carbon/NL.csv", "lat": 52.3676, "lon": 4.9041},
    {"name": "Brussel", "nodes": 4, "gpus_per_node": 4, "carbon-intensity-trace": "./carbon/BE.csv", "lat": 50.8503, "lon": 4.3517},
    {"name": "Berlin", "nodes": 2, "gpus_per_node": 6, "carbon-intensity-trace": "./carbon/DE.csv", "lat": 52.5200, "lon": 13.4050},
    {"name": "Rome", "nodes": 4, "gpus_per_node": 2, "carbon-intensity-trace": "./carbon/IT-CSO.csv", "lat": 41.9028, "lon": 12.4964},
    {"name": "Warsaw", "nodes": 6, "gpus_per_node": 4, "carbon-intensity-trace": "./carbon/PL.csv", "lat": 52.2297, "lon": 21.0122},
    {"name": "Paris", "nodes": 7, "gpus_per_node": 4, "carbon-intensity-trace": "./carbon/FR.csv", "lat": 48.8566, "lon": 2.3522},
    {"name": "London", "nodes": 8, "gpus_per_node": 4, "carbon-intensity-trace": "./carbon/GB.csv", "lat": 51.5074, "lon": -0.1278},
    {"name": "Copenhagen", "nodes": 2, "gpus_per_node": 6, "carbon-intensity-trace": "./carbon/DK-DK2.csv", "lat": 55.6761, "lon": 12.5683},
    {"name": "Oslo", "nodes": 1, "gpus_per_node": 6, "carbon-intensity-trace": "./carbon/NO-NO1.csv", "lat": 59.9139, "lon": 10.7522},
    {"name": "Bergen", "nodes": 3, "gpus_per_node": 2, "carbon-intensity-trace": "./carbon/NO-NO5.csv", "lat": 60.3913, "lon": 5.3221},
    {"name": "Budapest", "nodes": 2, "gpus_per_node": 2, "carbon-intensity-trace": "./carbon/HU.csv", "lat": 47.4979, "lon": 19.0402},
    {"name": "Oulu", "nodes": 4, "gpus_per_node": 2, "carbon-intensity-trace": "./carbon/FI.csv", "lat": 65.0121, "lon": 25.4651},
    {"name": "Athens", "nodes": 4, "gpus_per_node": 4, "carbon-intensity-trace": "./carbon/GR.csv", "lat": 37.9838, "lon": 23.7275},
    {"name": "Vilnius", "nodes": 1, "gpus_per_node": 4, "carbon-intensity-trace": "./carbon/LT.csv", "lat": 54.6872, "lon": 25.2797},
    {"name": "Riga", "nodes": 1, "gpus_per_node": 2, "carbon-intensity-trace": "./carbon/LV.csv", "lat": 56.9496, "lon": 24.1052},
    {"name": "Vaduz", "nodes": 1, "gpus_per_node": 1, "carbon-intensity-trace": "./carbon/LI.csv", "lat": 47.1410, "lon": 9.5209},
    {"name": "Luxembourg City", "nodes": 1, "gpus_per_node": 1, "carbon-intensity-trace": "./carbon/LU.csv", "lat": 49.6117, "lon": 6.1319},
    {"name": "Tallinn", "nodes": 1, "gpus_per_node": 6, "carbon-intensity-trace": "./carbon/EE.csv", "lat": 59.4370, "lon": 24.7535},
    {"name": "Zagreb", "nodes": 2, "gpus_per_node": 2, "carbon-intensity-trace": "./carbon/HR.csv", "lat": 45.8150, "lon": 15.9819},
    {"name": "Prague", "nodes": 5, "gpus_per_node": 2, "carbon-intensity-trace": "./carbon/CZ.csv", "lat": 50.0755, "lon": 14.4378},
    {"name": "Sofia", "nodes": 5, "gpus_per_node": 1, "carbon-intensity-trace": "./carbon/BG.csv", "lat": 42.6977, "lon": 23.3219},
    {"name": "Zurich", "nodes": 2, "gpus_per_node": 1, "carbon-intensity-trace": "./carbon/CH.csv", "lat": 47.3769, "lon": 8.5417},
    {"name": "Nicosia", "nodes": 1, "gpus_per_node": 1, "carbon-intensity-trace": "./carbon/CY.csv", "lat": 35.1856, "lon": 33.3823},
    {"name": "Vienna", "nodes": 1, "gpus_per_node": 6, "carbon-intensity-trace": "./carbon/AT.csv", "lat": 48.2082, "lon": 16.3738},
    {"name": "Reykjavik", "nodes": 1, "gpus_per_node": 2, "carbon-intensity-trace": "./carbon/IS.csv", "lat": 64.1466, "lon": -21.9426},
    {"name": "Bratislava", "nodes": 4, "gpus_per_node": 2, "carbon-intensity-trace": "./cognit/carbon/SK.csv", "lat": 48.1486, "lon": 17.1077},
    {"name": "Ljubljana", "nodes": 4, "gpus_per_node": 2, "carbon-intensity-trace": "./carbon/SI.csv", "lat": 46.0569, "lon": 14.5058},
    {"name": "Valletta", "nodes": 1, "gpus_per_node": 1, "carbon-intensity-trace": "./carbon/MT.csv", "lat": 35.8989, "lon": 14.5146},
    {"name": "Bucharest", "nodes": 4, "gpus_per_node": 1, "carbon-intensity-trace": "./carbon/RO.csv", "lat": 44.4268, "lon": 26.1025},
    {"name": "Dublin", "nodes": 4, "gpus_per_node": 2, "carbon-intensity-trace": "./carbon/IE.csv", "lat": 53.3498, "lon": -6.2603},
    {"name": "Trondheim", "nodes": 4, "gpus_per_node": 6, "carbon-intensity-trace": "./carbon/NO-NO3.csv", "lat": 63.4305, "lon": 10.3951},
    {"name": "Tromso", "nodes": 1, "gpus_per_node": 4, "carbon-intensity-trace": "./carbon/NO-NO4.csv", "lat": 69.6492, "lon": 18.9553},
    {"name": "Kristiansand", "nodes": 1, "gpus_per_node": 4, "carbon-intensity-trace": "./carbon/NO-NO2.csv", "lat": 58.1599, "lon": 8.0182}
]

In [8]:
import folium

# Initialize a map centered in Europe
m = folium.Map(location=[54.5260, 15.2551], zoom_start=4)

# Add markers for each cluster
for cluster in clusters:
    name = cluster["name"]
    lat = cluster["lat"]
    lon = cluster["lon"]
    nodes = cluster["nodes"]
    gpus = cluster["gpus_per_node"]
    carbon_trace = cluster["carbon-intensity-trace"]
    
    # Create a popup with cluster details
    popup_text = f"""
    <b>Cluster Name:</b> {name}<br>
    <b>Nodes:</b> {nodes}<br>
    <b>GPUs per Node:</b> {gpus}<br>
    <b>Carbon Intensity Trace:</b> {carbon_trace}
    """
    popup = folium.Popup(popup_text, max_width=300)
    
    # Add a marker
    folium.Marker(
        location=[lat, lon],
        popup=popup,
        tooltip=name,
        icon=folium.Icon(color="blue", icon="info-sign"),
    ).add_to(m)

# Display the map
m.save("clusters_map.html")
